In [27]:
# -*- coding: utf-8 -*-
# docker: {docker ps --filter "name=qdrant-test" --format "{{.Names}}" | grep -q qdrant-test && echo "✅ Qdrant már fut." || (docker ps -a --filter "name=qdrant-test" --format "{{.Names}}" | grep -q qdrant-test && (echo "▶️ Qdrant indítása..." && docker start qdrant-test) || (echo "🆕 Új Qdrant konténer létrehozása..." && docker run -d --name qdrant-test -p 6333:6333 -p 6334:6334 qdrant/qdrant && echo "✅ Qdrant elindítva. Nyisd meg: http://localhost:6333/dashboard"))}

# INVENTORYAI - Embedding teszt minden oszlopra több modellel
# Modellek: text-embedding-3-small, text-embedding-3-large, text-embedding-ada-002

import pandas as pd
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, PointStruct
from openai import OpenAI
import os
from dotenv import load_dotenv

# -------------------------------
# 1️⃣ .env betöltése és kulcs ellenőrzése
# -------------------------------
load_dotenv()  # .env fájl beolvasása
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("❌ Nincs beállítva az OPENAI_API_KEY környezeti változó! Ellenőrizd a .env fájlt.")

client_openai = OpenAI(api_key=api_key)

# -------------------------------
# 2️⃣ Excel beolvasása és szöveg tisztítás
# -------------------------------
df = pd.read_excel("../data/inventoryAI_data/leltar.XLSX")

# Minden string oszlopot kisbetűsre és whitespace-mentesre alakítunk
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.lower().str.strip()
    df[col] = df[col].str.replace(r'\s+', ' ', regex=True)

print("Adat előkészítés kész. Sorok száma:", len(df))

# -------------------------------
# 3️⃣ Qdrant kliens inicializálás
# -------------------------------
qdrant_client = QdrantClient(url="http://localhost:6333")


Adat előkészítés kész. Sorok száma: 49


## 1️⃣ Beállítások és adatok betöltése

Ez a rész:
- Betölti a `.env` fájlt, hogy az OpenAI API kulcsot használni tudjuk.
- Ellenőrzi, hogy az `OPENAI_API_KEY` környezeti változó be van-e állítva.
- Inicializálja az OpenAI klienst.
- Beolvassa az Excel leltáradatot és megtisztítja a szöveges oszlopokat (kisbetűs, felesleges szóközök eltávolítása).
- Inicializálja a Qdrant klienset.


In [28]:
# -------------------------------
# 4️⃣ Modellek listája
# -------------------------------
models = [
    {"name": "text-embedding-3-small", "size": 1536},
    {"name": "text-embedding-3-large", "size": 3072},  # text-embedding-3-large nagyobb vektor
    {"name": "text-embedding-ada-002", "size": 1536}
]

# -------------------------------
# 5️⃣ Embedding generálás és Qdrant feltöltés minden modellre
# -------------------------------
for model_info in models:
    model_name = model_info["name"]
    vector_size = model_info["size"]
    collection_name = f"leltar_{model_name.replace('-', '_')}_all_columns"

    print(f"\n--- Feldolgozás: {model_name} ---")

    # Ha a kollekció létezik, töröljük
    if qdrant_client.collection_exists(collection_name=collection_name):
        qdrant_client.delete_collection(collection_name=collection_name)

    # Kollekció létrehozása
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=vector_size, distance="Cosine")
    )

    points = []
    for idx, row in df.iterrows():
        # Minden oszlopot összefűzünk egy stringbe
        text = " | ".join([str(row[col]) for col in df.columns])

        # Embedding lekérése az OpenAI-tól
        response = client_openai.embeddings.create(
            model=model_name,
            input=text
        )
        vector = response.data[0].embedding

        # Qdrant PointStruct készítése
        point = PointStruct(
            id=int(row['Eszköz']),
            vector=vector,
            payload={col: row[col] for col in df.columns}
        )
        points.append(point)

    # Feltöltés a Qdrantba
    qdrant_client.upsert(
        collection_name=collection_name,
        points=points
    )
    print(f"{len(points)} vektor feltöltve a Qdrant kollekcióba: {collection_name}")

    # -------------------------------


--- Feldolgozás: text-embedding-3-small ---
49 vektor feltöltve a Qdrant kollekcióba: leltar_text_embedding_3_small_all_columns

--- Feldolgozás: text-embedding-3-large ---
49 vektor feltöltve a Qdrant kollekcióba: leltar_text_embedding_3_large_all_columns

--- Feldolgozás: text-embedding-ada-002 ---
49 vektor feltöltve a Qdrant kollekcióba: leltar_text_embedding_ada_002_all_columns


# 2️⃣ Modellek kiválasztása és embeddingek Qdrantba töltése

Ez a rész a következőket végzi:

- Meghatározza a használni kívánt modelleket:
  - `text-embedding-3-small`
  - `text-embedding-3-large`
  - `text-embedding-ada-002`
- Meghatározza a hozzájuk tartozó vektorméreteket.

## Folyamat

1. **Iterálás a modelleken:**
   - Minden modellen végigmegyünk, és létrehozunk egy külön Qdrant kollekciót az adott modellhez.
   - Ha a kollekció már létezik, törlésre kerül, hogy tiszta adatbázissal dolgozhassunk.
   - Kollekció létrehozása a Qdrantban:
     - Megadott vektorméret
     - `Cosine` távolság beállítás

2. **Adatok feldolgozása Excelből:**
   - Minden sorhoz összefűzi az összes oszlopot egyetlen szöveggé.
   - Lekéri az embeddinget az OpenAI modelltől.
   - Létrehoz egy `PointStruct` objektumot, ami tartalmazza:
     - ID
     - Vektor
     - Összes payload mező


    --
    **Megjegyzés**
    - Jelenleg minden sorhoz külön API-hívással készítjük az embeddingeket. Ez kis adatmennyiségnél (pl. 50 sor) teljesen megfelelő, pontos és könnyen követhető.
    - Ha a sorok száma növekszik, érdemes áttérni batch feldolgozásra, ahol egyszerre több sorhoz generálunk embeddinget. Ez gyorsabb és költséghatékonyabb, miközben minden sorhoz külön embeddinget kapunk, így a keresés pontossága nem csökken.

3. **Vektorok feltöltése:**
   - Az összegyűjtött vektorokat feltölti a Qdrant kollekcióba.
   - Kiírja, hogy hány vektor került feltöltésre az adott kollekcióba.

## Összegzés

Itt történik az adatok tényleges vektorokká alakítása és a Qdrant adatbázisba történő feltöltése minden kiválasztott modell alapján.



In [29]:
# 6️⃣ Több keresőszöveg tesztelése
query_texts = [
    "cisco callmanager licenc",
    "xilinx szoftver",
    "pc bér+kiegészítő program",
    "matlab 25 felhasználós"
]

for query_text in query_texts:
    # Ha ki akarjuk emelni egy oszlopot, pl. "Eszköz megnevezése", azt itt csinálhatjuk
    # weighted_text = f"{row['Eszköz megnevezése']} | {row['Alszám']} | ... "

    # Embedding lekérése OpenAI-tól (egy sor vagy batch is lehet)
    query_embedding = client_openai.embeddings.create(
        model=model_name,
        input=query_text
    ).data[0].embedding

    # Keresés Qdrantban a kiválasztott kollekcióban
    search_results = qdrant_client.query_points(
        collection_name=collection_name,
        query=query_embedding,
        limit=3,
        # distance='Cosine'  # ha más metrikát szeretnél, itt állítható (pl. Dot)
    )

    print(f"\nTop 3 releváns találat a keresőszövre: '{query_text}'")
    for hit in search_results.points:
        print(f"ID: {hit.id}, Score: {hit.score:.4f}")
        for col, val in hit.payload.items():
            print(f"   {col}: {val}")




Top 3 releváns találat a keresőszövre: 'cisco callmanager licenc'
ID: 1000222, Score: 0.8600
   Eszköz: 1000222
   Alszám: 0
   Aktiválás dátuma: 2021-08-01T00:00:00
   Eszköz megnevezése: sw-ccme-ul-7940 cisco callmanager licenc
   Besz.ért.: 0
   Kum. ÉCS: 0
   K.sz.ért: 0
   Pénznem: huf
   Mennyiség: 1
   Leltárszám: 70667
   Telephely: nikszf4421
ID: 1000223, Score: 0.8596
   Eszköz: 1000223
   Alszám: 0
   Aktiválás dátuma: 2021-08-01T00:00:00
   Eszköz megnevezése: sw-ccme-ul-7940 cisco callmanager licenc
   Besz.ért.: 0
   Kum. ÉCS: 0
   K.sz.ért: 0
   Pénznem: huf
   Mennyiség: 1
   Leltárszám: 70668
   Telephely: nikszf4421
ID: 1000221, Score: 0.8582
   Eszköz: 1000221
   Alszám: 0
   Aktiválás dátuma: 2021-08-01T00:00:00
   Eszköz megnevezése: sw-ccme-ul-7940 cisco callmanager licenc
   Besz.ért.: 0
   Kum. ÉCS: 0
   K.sz.ért: 0
   Pénznem: huf
   Mennyiség: 1
   Leltárszám: 70666
   Telephely: nikszf4421

Top 3 releváns találat a keresőszövre: 'xilinx szoftver'
ID: 1000000

# Qdrant + OpenAI Embedding Teszt

## Mi történik a kódban

1. **Keresőszövegek definiálása**
   Egy lista keresőszöveget tartalmaz, amire a Qdrant adatbázisban releváns találatokat keresünk.

2. **Embedding generálás**
   Minden keresőszöveget az OpenAI kiválasztott embedding modellje vektorrá alakít.

3. **Keresés a Qdrant kollekcióban**
   A lekérdezett embeddinget használva a Qdrant top 3 releváns találatot ad vissza.

4. **Eredmények kiírása**
   A találatok azonosítója, relevancia pontszáma és a kapcsolódó adatok kerülnek megjelenítésre.

## Konklúzió

A három tesztelt modell közül a **`text-embedding-3-large`** teljesített a legjobban.
Ennek oka, hogy a nagyobb, 3072 dimenziós vektorok pontosabban reprezentálják a keresőszöveg és az adatbázis tartalmát, így finomabb és relevánsabb találatokat adnak.
A `text-embedding-3-small` gyors és elfogadható, míg az `ada-002` modell régebbi, és pontosságban elmarad a két újabb modelltől.
